In [34]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import sys
sys.path.append('Desktop/J-league/helper_functions')
from helper_functions.j_leagueAPI import J_leagueAPI
from helper_functions.get_mins import get_mins
from scipy.spatial.distance import euclidean
import numpy as np
import helper_functions.stats_functions as sf

# TO DO #
### Attackers/Midfielders ##
- Create a normalized creative passing stat
- Create a normalized physical (speed/intensity) stat
- Create a normalized endurance stat
- Create a normalized carrying stat
- Create a normalized dribbling stat
- Create a normalized ball retention under pressure stat
- Create a Progressive passes under pressure

### Defenders ###
- Create a normalized physical (speed/intensity) stat
- Create a normalized endurance stat
- get average line height

### Creative Passing Stats ###
- PPA, Under Pressure, pass.assisted_shot_id, 
- pass.goal_assist, pass.through_ball
- pass.pass_success_probability, pass.type.name
- pass.pass_cluster_label, pass.pass_cluster_probability
- pass.shot_assist, pass.inswinging, pass.cross
- pass.goal_assist, pass.through_ball
- pass.technique.name
- progressive passes/progressive passes under pressure
- progressive passes played within 2 seconds of recieving the ball

# Dashboard Project
Midfield Template
- xG
- Goals
- Passing OBV
- PPA
- Progressive Passes
- Assists
- xA
- CPA
- Carry OBV
- Dribbles
- Dribble success
- tackles
- interceptions
- tackle success

Forward Template
- xG
- xG - PsxG (shot.gk_save_difficulty_xg)
- Goals
- Assists
- xA
- PPA
- CPA
- Dribbles
- Dribble success
- tackles
- interceptions
- tackle success

In [39]:
pd.set_option('display.max_columns', None)

In [41]:
# Load Physical Data
api = J_leagueAPI()
api.load_data('data/hudl_physical.json')
physical = api.extract_local_network()

In [42]:
# Load Matches Data
matches = pd.read_json('data/sb_matches.json')

In [43]:
# Load Event Data
events = pd.read_json('data/sb_events.json')

In [46]:
# List of all stats
stats = []

In [47]:
minutes_df = get_mins(events)
stats.append(minutes_df)

/Users/komdean/Desktop/J-league/helper_functions/get_mins.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minutes_played['timestamp'] = minutes_played['timestamp'].astype(str).str.split().str[1]
/Users/komdean/Desktop/J-league/helper_functions/get_mins.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minutes_played['timestamp'] = pd.to_timedelta(minutes_played['timestamp'])
/Users/komdean/Desktop/J-league/helper_functions/get_mins.py:19: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [48]:
# Create Progressive passes Dataframe
pp_df = sf.progressive_passes(events)

# Append to stats
stats.append(pp_df)

In [49]:
# Create Progressive passes under pressure Dataframe
under_pressure_df = events[events['under_pressure'] == 1]
up_pp_df = sf.progressive_passes(under_pressure_df)
up_pp_df = up_pp_df.rename(columns={'progressive_passes' : 'progressive_passes_up'})

# Append to stats
stats.append(up_pp_df)

/Users/komdean/Desktop/J-league/helper_functions/stats_functions.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x'] = df['location'].str[0]
/Users/komdean/Desktop/J-league/helper_functions/stats_functions.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y'] = df['location'].str[1]
/Users/komdean/Desktop/J-league/helper_functions/stats_functions.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [50]:
# Create Carries dataframe
carry_df = events[events['type.name'] == 'Carry']
carry_df['carry_end_x'] = carry_df['carry.end_location'].str[0]
carry_df['carry_end_y'] = carry_df['carry.end_location'].str[1]
carry_df['carry_distance'] = np.sqrt((carry_df['x'] + carry_df['carry_end_x']) ** 2 + 
                                     (carry_df['y'] + carry_df['carry_end_y']) ** 2)
carry_obv = carry_df.groupby(['player.id', 'player.name'])['obv_total_net'].sum().reset_index(name='carry_obv')
carry_distance = carry_df.groupby(['player.id', 'player.name'])['carry_distance'].sum().reset_index()
carry_total = carry_df.groupby(['player.id', 'player.name']).size().reset_index(name='total_carries')
carries = pd.merge(carry_obv, carry_total, how='outer', on=['player.id', 'player.name'])
carries = pd.merge(carries, carry_distance, how='outer', on=['player.id', 'player.name'])
carries['distance_per_carry'] = carries['carry_distance'] / carries['total_carries']
carries['obv_per_carry'] = carries['carry_obv'] / carries['total_carries']

# Append to Stats
stats.append(carries)

/var/folders/rr/7hqln5dd7_55vxvrgjhmz1gw0000gn/T/ipykernel_26399/4252574311.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  carry_df['carry_end_x'] = carry_df['carry.end_location'].str[0]
/var/folders/rr/7hqln5dd7_55vxvrgjhmz1gw0000gn/T/ipykernel_26399/4252574311.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  carry_df['carry_end_y'] = carry_df['carry.end_location'].str[1]
/var/folders/rr/7hqln5dd7_55vxvrgjhmz1gw0000gn/T/ipykernel_26399/4252574311.py:5: SettingWithCopyWarning: 
A value is trying to 

In [51]:
# Create total passes dataframe
passes_df = events[events['type.name'] == 'Pass']
successful_passes_df = passes_df[passes_df['pass.outcome.name'].isna()]
total_passes = passes_df.groupby(['player.id', 'player.name']).size().reset_index(name='total_passes')
successful_passes = successful_passes_df.groupby(['player.id', 'player.name']).size().reset_index(name='successful_passes')
pass_count = pd.merge(total_passes, successful_passes, how='outer', on=['player.id', 'player.name'])

# Append to stats
stats.append(pass_count)

In [52]:
# Create xG dataframe for shooting with feet
shots_df = events[events['type.name'] == 'Shot']

# Sum up xG
xg_df = shots_df[['player.id', 'player.name', 'shot.statsbomb_xg', 'shot.shot_execution_xg_uplift']]
xg_df = xg_df.groupby(['player.id', 'player.name']).sum().reset_index()

# Sum up shots and goals
goals_df = shots_df[shots_df['shot.outcome.name'] == 'Goal']
goals_df = goals_df.groupby(['player.id', 'player.name']).size().reset_index(name='goals')
shots_df = shots_df.groupby(['player.id', 'player.name']).size().reset_index(name='shots')

# Merge
shots = pd.merge(goals_df, shots_df, how='outer', on=['player.id', 'player.name'])
shots = pd.merge(shots, xg_df, how='outer', on=['player.id', 'player.name'])

# Append to stats
stats.append(shots)

In [53]:
# Merge Dataframes
stats_df = pd.merge(stats[0], stats[1], how='outer', on=['player.id'])
for stat in stats[2:]:
    stats_df = pd.merge(stats_df, stat, how='outer', on=['player.id', 'player.name'])
stats_df.sort_values(by='playtime_minutes', ascending=True, inplace=True)
stats_df[:50]

,player.id,playtime_minutes,player.name,progressive_passes,progressive_passes_up,carry_obv,total_carries,carry_distance,distance_per_carry,obv_per_carry,total_passes,successful_passes,goals,shots,shot.statsbomb_xg,shot.shot_execution_xg_uplift
512,379894,95.293000,Shion Inoue,193,27.0,0.015300,49.0,5160.682310,105.320047,0.000312,65.0,53.0,NaN,1.0,0.0310,-0.0310
407,100717,95.398117,Teppei Yachida,162,32.0,0.079400,35.0,6363.658783,181.818822,0.002269,52.0,34.0,NaN,NaN,NaN,NaN
93,37223,95.459117,Shintaro Kurumaya,317,22.0,0.096400,88.0,9980.852634,113.418780,0.001095,108.0,102.0,NaN,1.0,0.1102,-0.1102
560,426942,96.276300,Yuto Anzai,290,87.0,0.393400,67.0,11172.597235,166.755183,0.005872,49.0,35.0,1.0,7.0,0.8474,0.1626
195,38539,96.498050,Tatsuya Tanaka,47,3.0,0.013100,8.0,1907.483810,238.435476,0.001638,21.0,18.0,NaN,NaN,NaN,NaN
534,396525,97.090533,Hisatsugu Ishii,627,181.0,0.623200,143.0,23967.597257,167.605575,0.004358,114.0,81.0,1.0,15.0,0.7160,-0.3916
94,37227,97.180250,Kazuya Yamamura,289,69.0,0.099495,76.0,10873.273477,143.069388,0.001309,87.0,73.0,NaN,2.0,0.1345,-0.1345
244,42594,97.279800,Gakuto Notsuda,164,34.0,0.035700,28.0,4417.208294,157.757439,0.001275,52.0,40.0,NaN,1.0,0.0235,-0.0235
321,95275,97.596533,Daiki Ogawa,134,38.0,0.101500,33.0,5758.766824,174.508086,0.003076,49.0,35.0,NaN,NaN,NaN,NaN
363,96074,97.596533,Masatoshi Ishida,147,46.0,0.036400,21.0,3386.143465,161.244927,0.001733,22.0,17.0,NaN,2.0,0.0523,-0.0472


In [54]:
events.head()

,id,index,period,timestamp,minute,second,possession,duration,related_events,location,type.id,type.name,possession_team.id,possession_team.name,play_pattern.id,play_pattern.name,team.id,team.name,tactics.formation,tactics.lineup,pass.end_location,carry.end_location,shot.end_location,shot.freeze_frame,goalkeeper.end_location,match_id,obv_for_after,obv_for_before,obv_for_net,obv_against_after,obv_against_before,obv_against_net,obv_total_net,player.id,player.name,position.id,position.name,pass.length,pass.angle,pass.pass_success_probability,pass.recipient.id,pass.recipient.name,pass.height.id,pass.height.name,pass.body_part.id,pass.body_part.name,pass.type.id,pass.type.name,pass.pass_cluster_id,pass.pass_cluster_label,pass.pass_cluster_probability,under_pressure,pass.aerial_won,pass.outcome.id,pass.outcome.name,duel.type.id,duel.type.name,clearance.right_foot,clearance.body_part.id,clearance.body_part.name,clearance.head,ball_receipt.outcome.id,ball_receipt.outcome.name,pass.switch,out,counterpress,shot.statsbomb_xg,shot.shot_execution_xg,shot.shot_execution_xg_uplift,shot.gk_positioning_xg_suppression,shot.technique.id,shot.technique.name,shot.body_part.id,shot.body_part.name,shot.type.id,shot.type.name,shot.outcome.id,shot.outcome.name,goalkeeper.position.id,goalkeeper.position.name,goalkeeper.type.id,goalkeeper.type.name,clearance.aerial_won,goalkeeper.body_part.id,goalkeeper.body_part.name,goalkeeper.outcome.id,goalkeeper.outcome.name,interception.outcome.id,interception.outcome.name,duel.outcome.id,duel.outcome.name,shot.first_time,off_camera,pass.assisted_shot_id,pass.goal_assist,pass.through_ball,pass.technique.id,pass.technique.name,shot.one_on_one,shot.gk_save_difficulty_xg,shot.gk_shot_stopping_xg_suppression,shot.key_pass_id,goalkeeper.technique.id,goalkeeper.technique.name,dribble.outcome.id,dribble.outcome.name,ball_recovery.recovery_failure,foul_committed.card.id,foul_committed.card.name,foul_won.defensive,pass.shot_assist,pass.inswinging,pass.cross,injury_stoppage.in_chain,substitution.outcome.id,substitution.outcome.name,substitution.replacement.id,substitution.replacement.name,clearance.left_foot,pass.no_touch,shot.saved_off_target,goalkeeper.shot_saved_off_target,50_50.outcome.id,50_50.outcome.name,pass.cut_back,block.offensive,foul_committed.advantage,foul_won.advantage,pass.miscommunication,dribble.nutmeg,pass.deflected,block.deflection,pass.outswinging,clearance.other,bad_behaviour.card.id,bad_behaviour.card.name,foul_committed.type.id,foul_committed.type.name,dribble.overrun,block.save_block,miscontrol.aerial_won,shot.aerial_won,foul_committed.offensive,pass.straight,ball_recovery.offensive,shot.deflected,shot.open_goal,goalkeeper.punched_out,foul_committed.penalty,foul_won.penalty,shot.shot_shot_assist,dribble.no_touch,shot.saved_to_post,goalkeeper.shot_saved_to_post,shot.shot_goal_assist,shot.follows_dribble,goalkeeper.lost_out,goalkeeper.lost_in_play,player_off.permanent,shot.redirect,goalkeeper.success_out,goalkeeper.success_in_play,x,y,pass_end_x,pass_end_y,is_progressive
0,dc1f3c18-004f-45d9-8f86-71d670f9cd97,1,1,2025-03-13 00:00:00.000,0,0,1,0.0000,None,None,35,Starting XI,1881,Kashima Antlers,1,Regular Play,1881,Kashima Antlers,442.0,"[{'jersey_number': 1, 'player.id': 126936, 'pl...",None,None,None,None,None,3925601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,8e81e4fd-fc8c-4efb-b9c4-6d2a55008d43,2,1,2025-03-13 00:00:00.000,0,0,1,0.0000,None,None,35,Starting XI,1881,Kashima Antlers,1,Regu

In [55]:
set(events['type.name'])

{'50/50',
 'Bad Behaviour',
 'Ball Receipt*',
 'Ball Recovery',
 'Block',
 'Carry',
 'Clearance',
 'Dispossessed',
 'Dribble',
 'Dribbled Past',
 'Duel',
 'Error',
 'Foul Committed',
 'Foul Won',
 'Goal Keeper',
 'Half End',
 'Half Start',
 'Injury Stoppage',
 'Interception',
 'Miscontrol',
 'Offside',
 'Own Goal Against',
 'Own Goal For',
 'Pass',
 'Player Off',
 'Player On',
 'Pressure',
 'Referee Ball-Drop',
 'Shield',
 'Shot',
 'Starting XI',
 'Substitution',
 'Tactical Shift'}

In [56]:
for team in set(events['team.id']):
    if len(set(events[events['team.id'] == team]['match_id'])) == 37:
        print(set(events[events['team.id'] == team]['team.name']))

{'Avispa Fukuoka'}
{'Júbilo Iwata'}
{'Vissel Kobe'}
{'Kashiwa Reysol'}
{'Kawasaki Frontale'}
{'Sagan Tosu'}
{'Shonan Bellmare'}
{'Consadole Sapporo'}
